In [4]:
from sympy.physics.mechanics import *
from sympy import symbols, sin, cos, atan, Matrix
q1, q2 = dynamicsymbols('q1:3')
q1d, q2d = dynamicsymbols('q1:3', level=1)
L, m, g, t = symbols('L, m, g, t')



# Compose World Frame
N = ReferenceFrame('N')
pN = Point('N*')
pN.set_vel(N, 0)

# A.x is along the pendulum
A = N.orientnew('A', 'axis', [q1, N.z])

# Create point P, the pendulum mass
P = pN.locatenew('P1', L * sin(q1) * N.x + L * cos(q1) * N.y)
P.set_vel(N, P.pos_from(pN).dt(N))
pP = Particle('pP', P, m)

# Calculate the lagrangian, and form the equations of motion
L = Lagrangian(N, pP)
LM = LagrangesMethod(L, [q1], forcelist=[(P, m*g*N.x)], frame=N)
lag_eqs = LM.form_lagranges_equations()




q = mechanics.dynamicsymbols(f'q:{n}')
p = mechanics.dynamicsymbols(f'p:{n}')

m = symbols(f'm:{n}')
l = symbols(f'l:{n}')
g, t = symbols('g,t')
    